In [1]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [4]:
spark.version

'3.0.3'

In [ ]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv

```bash
!wc -l fhvhv_tripdata_2021-01.csv
11908469 fhvhv_tripdata_2021-01.csv
```

In [29]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [30]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true)))

```bash
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv
```

In [31]:
import pandas as pd

In [32]:
df_pandas = pd.read_csv('head.csv')

In [33]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [34]:
spark.createDataFrame(df_pandas).schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

In [17]:
from pyspark.sql import types

In [18]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [19]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [20]:
df = df.repartition(24)

In [21]:
df.write.parquet('fhvhv/2021/01/')

In [35]:
df = spark.read.parquet('fhvhv/2021/01/')

In [36]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



```sql
SELECT * FROM df WHERE hvfhs_license_num = HV0003
```

In [37]:
from pyspark.sql import functions as F

In [38]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2021-01-02 17:05:47|2021-01-02 17:21:03|          42|         166|   null|
|           HV0003|              B02765|2021-01-04 13:04:14|2021-01-04 13:35:32|         188|         170|   null|
|           HV0005|              B02510|2021-01-02 20:43:11|2021-01-02 20:52:44|         231|         158|   null|
|           HV0003|              B02764|2021-01-01 04:20:52|2021-01-01 04:45:01|         145|         265|   null|
|           HV0003|              B02878|2021-01-01 01:55:17|2021-01-01 02:15:29|          26|         188|   null|
|           HV0005|              B02510|2021-01-01 05:38:34|2021-01-01 05:46:46|

In [39]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [40]:
crazy_stuff('B02884')

's/b44'

In [41]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [42]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/9ce| 2021-01-02|  2021-01-02|          42|         166|
|  s/acd| 2021-01-04|  2021-01-04|         188|         170|
|  e/9ce| 2021-01-02|  2021-01-02|         231|         158|
|  e/acc| 2021-01-01|  2021-01-01|         145|         265|
|  e/b3e| 2021-01-01|  2021-01-01|          26|         188|
|  e/9ce| 2021-01-01|  2021-01-01|         198|          80|
|  e/9ce| 2021-01-03|  2021-01-03|          40|          21|
|  a/b43| 2021-01-01|  2021-01-01|         141|         265|
|  s/b44| 2021-01-03|  2021-01-03|         242|          18|
|  e/b47| 2021-01-05|  2021-01-05|          76|          76|
|  e/b3c| 2021-01-01|  2021-01-01|           7|          82|
|  e/9ce| 2021-01-03|  2021-01-03|         152|         138|
|  a/a7a| 2021-01-02|  2021-01-02|         133|          89|
|  e/9ce| 2021-01-02|  2

In [43]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003')

DataFrame[pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int]